# 🚀 Descarga Masiva de Dataset para YOLOv8

**Proyecto Integrador - IFTS24**

Este notebook descarga automáticamente imágenes de Google/Bing y genera las anotaciones YOLO correspondientes.

## 🎯 Objetivo
- Descargar imágenes masivamente por clase (producto)
- Generar anotaciones automáticas en formato YOLO
- Crear dataset listo para entrenamiento de YOLOv8

## 📦 Clases a Descargar
- coca-cola
- sprite
- fanta
- pepsi
- seven-up
- bottle (genérico)

## ⚠️ Importante
- Ejecuta celda por celda
- Las descargas pueden tomar tiempo
- Verifica que tengas suficiente espacio en disco

In [ ]:
# Instalar dependencias necesarias
!pip install bing-image-downloader
!pip install icrawler
!pip install Pillow
!pip install tqdm

# Importar librerías
import os
import shutil
from pathlib import Path
from bing_image_downloader import downloader
from icrawler.builtin import BingImageCrawler
from PIL import Image
import yaml
from tqdm import tqdm
import random

print("✅ Dependencias instaladas")

In [ ]:
# Configuración del dataset
CLASES = {
    0: 'coca-cola',
    1: 'sprite',
    2: 'fanta',
    3: 'pepsi',
    4: 'seven-up',
    5: 'bottle'
}

# Configuración de descarga
IMAGENES_POR_CLASE = 500  # Máximo 500 por clase
IMAGENES_POR_CLASE_ENTRENAMIENTO = 350
IMAGENES_POR_CLASE_VALIDACION = 100
IMAGENES_POR_CLASE_TEST = 50

print(f"📊 Clases configuradas: {len(CLASES)}")
print(f"📸 Imágenes por clase: {IMAGENES_POR_CLASE}")
print(f"🎯 Distribución: {IMAGENES_POR_CLASE_ENTRENAMIENTO} train, {IMAGENES_POR_CLASE_VALIDACION} val, {IMAGENES_POR_CLASE_TEST} test")

In [ ]:
# Crear estructura de directorios
def crear_estructura_dataset():
    """Crea la estructura de directorios para el dataset"""
    dirs = [
        'datasets/images/train',
        'datasets/images/val',
        'datasets/images/test',
        'datasets/labels/train',
        'datasets/labels/val',
        'datasets/labels/test',
        'temp_downloads'  # Directorio temporal para descargas
    ]
    
    for dir_path in dirs:
        os.makedirs(dir_path, exist_ok=True)
        print(f"📁 Creado: {dir_path}")
    
    print("✅ Estructura de dataset creada")

crear_estructura_dataset()

In [ ]:
# Función para descargar imágenes con Bing
def descargar_imagenes_bing(clase_nombre, num_imagenes=100):
    """Descarga imágenes usando Bing Image Downloader"""
    try:
        # Crear directorio temporal para esta clase
        temp_dir = f"temp_downloads/{clase_nombre}"
        os.makedirs(temp_dir, exist_ok=True)
        
        # Búsqueda inteligente para productos
        query = f"{clase_nombre} bottle product front view isolated white background"
        
        print(f"🔍 Buscando: {query}")
        
        # Descargar imágenes
        downloader.download(
            query,
            limit=num_imagenes,
            output_dir=temp_dir,
            adult_filter_off=True,
            force_replace=False,
            timeout=60,
            verbose=False
        )
        
        # Contar imágenes descargadas
        imagenes = list(Path(temp_dir).glob("*.jpg")) + list(Path(temp_dir).glob("*.png")) + list(Path(temp_dir).glob("*.jpeg"))
        print(f"✅ Descargadas {len(imagenes)} imágenes para {clase_nombre}")
        
        return imagenes
        
    except Exception as e:
        print(f"❌ Error descargando {clase_nombre}: {e}")
        return []

In [ ]:
# Función para filtrar imágenes válidas
def filtrar_imagenes_validas(imagenes_paths):
    """Filtra imágenes que se pueden abrir y tienen buen tamaño"""
    imagenes_validas = []
    
    for img_path in imagenes_paths:
        try:
            with Image.open(img_path) as img:
                # Verificar tamaño mínimo
                if img.size[0] >= 224 and img.size[1] >= 224:
                    imagenes_validas.append(img_path)
        except Exception:
            continue
    
    print(f"✅ {len(imagenes_validas)} imágenes válidas de {len(imagenes_paths)} totales")
    return imagenes_validas

In [ ]:
# Función para generar anotaciones YOLO
def generar_anotacion_yolo(imagen_path, clase_id, output_path):
    """Genera anotación YOLO para una imagen completa (bounding box completo)"""
    try:
        with Image.open(imagen_path) as img:
            width, height = img.size
            
            # Bounding box que cubre toda la imagen
            # Formato YOLO: clase_id x_center y_center width height (normalizado 0-1)
            x_center = 0.5  # Centro horizontal
            y_center = 0.5  # Centro vertical
            bbox_width = 1.0  # Ancho completo
            bbox_height = 1.0  # Alto completo
            
            # Crear línea de anotación
            anotacion = f"{clase_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n"
            
            # Guardar anotación
            with open(output_path, 'w') as f:
                f.write(anotacion)
            
            return True
    except Exception as e:
        print(f"❌ Error generando anotación para {imagen_path}: {e}")
        return False

In [ ]:
# Función para organizar dataset
def organizar_dataset(clase_id, clase_nombre, imagenes_paths):
    """Organiza imágenes y anotaciones en estructura train/val/test"""
    
    # Mezclar imágenes aleatoriamente
    random.shuffle(imagenes_paths)
    
    # Dividir en train/val/test
    n_total = len(imagenes_paths)
    n_train = min(IMAGENES_POR_CLASE_ENTRENAMIENTO, n_total * 70 // 100)
    n_val = min(IMAGENES_POR_CLASE_VALIDACION, n_total * 20 // 100)
    n_test = min(IMAGENES_POR_CLASE_TEST, n_total - n_train - n_val)
    
    splits = {
        'train': imagenes_paths[:n_train],
        'val': imagenes_paths[n_train:n_train+n_val],
        'test': imagenes_paths[n_train+n_val:n_train+n_val+n_test]
    }
    
    print(f"📊 {clase_nombre}: {n_train} train, {n_val} val, {n_test} test")
    
    # Procesar cada split
    contador_imagen = 0
    for split_name, split_imagenes in splits.items():
        for img_path in split_imagenes:
            try:
                # Nombre base para imagen y anotación
                base_name = f"{clase_nombre}_{contador_imagen:04d}"
                
                # Copiar imagen
                img_destino = f"datasets/images/{split_name}/{base_name}.jpg"
                shutil.copy2(img_path, img_destino)
                
                # Generar anotación
                label_destino = f"datasets/labels/{split_name}/{base_name}.txt"
                generar_anotacion_yolo(img_destino, clase_id, label_destino)
                
                contador_imagen += 1
                
            except Exception as e:
                print(f"❌ Error procesando {img_path}: {e}")
                continue
    
    print(f"✅ {clase_nombre}: Procesadas {contador_imagen} imágenes")
    return contador_imagen

In [ ]:
# Función principal de descarga
def descargar_dataset_completo():
    """Descarga el dataset completo para todas las clases"""
    print("🚀 INICIANDO DESCARGA MASIVA DE DATASET")
    print("="*50)
    
    total_imagenes = 0
    
    for clase_id, clase_nombre in CLASES.items():
        print(f"\n📦 Procesando clase: {clase_nombre} (ID: {clase_id})")
        print("-"*40)
        
        # Descargar imágenes
        imagenes_raw = descargar_imagenes_bing(clase_nombre, IMAGENES_POR_CLASE)
        
        if not imagenes_raw:
            print(f"⚠️ No se pudieron descargar imágenes para {clase_nombre}")
            continue
        
        # Filtrar imágenes válidas
        imagenes_validas = filtrar_imagenes_validas(imagenes_raw)
        
        if not imagenes_validas:
            print(f"⚠️ No hay imágenes válidas para {clase_nombre}")
            continue
        
        # Organizar en dataset
        imagenes_procesadas = organizar_dataset(clase_id, clase_nombre, imagenes_validas)
        total_imagenes += imagenes_procesadas
        
        print(f"✅ {clase_nombre}: {imagenes_procesadas} imágenes procesadas")
    
    print("\n" + "="*50)
    print(f"🎉 DATASET COMPLETADO: {total_imagenes} imágenes totales")
    print("📁 Estructura creada en carpeta 'datasets/'")
    print("🚀 Listo para entrenamiento con: python train_yolo_model.py")
    print("="*50)

# Ejecutar descarga completa
descargar_dataset_completo()

In [ ]:
# Crear archivo data.yaml para YOLOv8
def crear_data_yaml():
    """Crea el archivo de configuración data.yaml"""
    data_config = {
        'path': './datasets',
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'names': CLASES,
        'nc': len(CLASES)
    }
    
    with open('datasets/data.yaml', 'w') as f:
        yaml.dump(data_config, f, default_flow_style=False)
    
    print("✅ Archivo data.yaml creado")
    print("📄 Contenido:")
    with open('datasets/data.yaml', 'r') as f:
        print(f.read())

crear_data_yaml()

In [ ]:
# Verificar dataset creado
def verificar_dataset():
    """Verifica que el dataset se creó correctamente"""
    print("🔍 Verificando dataset creado...")
    
    # Contar archivos por directorio
    dirs_to_check = [
        'datasets/images/train',
        'datasets/images/val',
        'datasets/images/test',
        'datasets/labels/train',
        'datasets/labels/val',
        'datasets/labels/test'
    ]
    
    total_imagenes = 0
    total_labels = 0
    
    for dir_path in dirs_to_check:
        if os.path.exists(dir_path):
            archivos = os.listdir(dir_path)
            print(f"📁 {dir_path}: {len(archivos)} archivos")
            
            if 'images' in dir_path:
                total_imagenes += len(archivos)
            else:
                total_labels += len(archivos)
        else:
            print(f"❌ {dir_path}: NO EXISTE")
    
    print(f"\n📊 RESUMEN:")
    print(f"  🖼️ Imágenes totales: {total_imagenes}")
    print(f"  🏷️ Etiquetas totales: {total_labels}")
    print(f"  📦 Dataset listo: {'✅ SÍ' if total_imagenes > 0 and total_labels > 0 else '❌ NO'}")

verificar_dataset()

In [ ]:
# Crear archivo ZIP para descarga
def crear_zip_dataset():
    """Crea un archivo ZIP con el dataset completo"""
    try:
        shutil.make_archive('dataset_yolo_productos', 'zip', 'datasets')
        print("✅ Dataset comprimido: dataset_yolo_productos.zip")
        print("📥 Descárgalo desde el panel izquierdo (Files)")
    except Exception as e:
        print(f"❌ Error creando ZIP: {e}")

crear_zip_dataset()

# 🎉 ¡DATASET COMPLETADO!

## 📋 Resumen
- ✅ Imágenes descargadas de Bing/Google
- ✅ Anotaciones YOLO generadas automáticamente
- ✅ Dataset organizado en train/val/test
- ✅ Archivo data.yaml creado
- ✅ Dataset comprimido para descarga

## 🚀 Próximos Pasos

1. **Descarga el ZIP**: `dataset_yolo_productos.zip`
2. **Extrae en tu proyecto** local
3. **Entrena el modelo**:
   ```bash
   python train_yolo_model.py
   ```
4. **Usa en la aplicación**:
   ```bash
   streamlit run app.py
   ```

## 💡 Consejos
- Si el dataset es pequeño, ejecuta el notebook varias veces
- Puedes modificar las clases en la configuración inicial
- Para mejores resultados, combina con imágenes propias

**¡Tu dataset de productos está listo para entrenar YOLOv8!** 🚀